In [2]:
import torch
import numpy
import sklearn

# When there is no missing values in the input time series

In [6]:
x = torch.randn(1, 1, 5)
print("Assume we input a time series, [batchSize= 1, column number = 1, length of the series = 5]")
x

Assume we input a time series, [batchSize= 1, column number = 1, length of the series = 5]


tensor([[[-1.2694,  0.9239, -0.6867,  0.8838,  0.0558]]])

In [5]:
def chomp1D(x, chomp_size):
    return x[:, :, :-chomp_size]

Note: since weightnorm and relu activation function do not change shape, we omit them in this testing code for better clarity.

In [12]:
print("There are several Conv1 layers in the whole network. All of the conv1's out_channels=40 except the last conv1's out_channel=320.")

conv1 = torch.nn.Conv1d(in_channels=1, out_channels=320, kernel_size=3, padding=2, dilation=1)
conv1

There are several Conv1 layers in the whole network. All of the conv1's out_channels=40 except the last conv1's out_channel=320.


Conv1d(1, 320, kernel_size=(3,), stride=(1,), padding=(2,))

In [7]:
output_conv1 = conv1(x)
print(output_conv1.shape)
output_conv1

torch.Size([1, 320, 7])


tensor([[[-0.2628, -0.5828,  0.4189,  ...,  0.3485, -0.3102, -0.1636],
         [-0.0229, -0.2682,  0.5610,  ...,  0.4368, -0.1745,  0.0161],
         [ 0.4144,  0.7051, -0.1756,  ...,  0.1367,  0.8880,  0.5366],
         ...,
         [ 0.0023, -0.6713, -0.5978,  ..., -0.5608,  0.0290, -0.3106],
         [-0.3060, -0.2412, -0.8485,  ..., -0.7783, -0.2383, -0.4141],
         [ 0.2669, -0.2945,  0.3951,  ..., -0.2489, -0.8139, -0.3367]]],
       grad_fn=<SqueezeBackward1>)

In [168]:
output_chomp1D = chomp1D(output_conv1, 2)
print(output_chomp1D.shape)
output_chomp1D

torch.Size([1, 320, 5])


tensor([[[-0.1731, -0.0301, -0.2691,  0.2169,  0.7169],
         [ 0.2299, -0.4404, -0.4675,  0.3638, -0.0301],
         [ 0.1225,  0.3330, -0.3448, -0.5038,  0.6929],
         ...,
         [-0.2150,  0.1279,  0.5057,  0.0972, -0.2664],
         [-0.0068,  0.1445, -0.5320, -0.8261,  0.3189],
         [ 0.7271,  1.1957,  0.6107, -0.0762,  1.1329]]],
       grad_fn=<SliceBackward>)

In [171]:
reduce_size = torch.nn.AdaptiveMaxPool1d(1)

output_maxpool = reduce_size(output_chomp1D)

print(output_maxpool.shape)
print("As we can see, each input single value turn into 320 values, and these are the important features of this single value. The length of the input series is disregarded, ie 5 becomes 1. So now the input series is represented by 1 number that has 320 features.")
output_maxpool

torch.Size([1, 320, 1])
As we can see, each input single value turn into 320 values, and these are the important features of this single value. The length of the input series is disregarded, ie 5 becomes 1. So now the input series is represented by 1 number that has 320 features.


tensor([[[ 7.1693e-01],
         [ 3.6379e-01],
         [ 6.9294e-01],
         [ 7.1109e-01],
         [ 4.5150e-01],
         [ 5.8228e-01],
         [ 7.4167e-01],
         [-1.8462e-02],
         [ 3.5803e-01],
         [ 1.2853e-02],
         [ 8.3715e-01],
         [ 9.6151e-01],
         [ 1.3426e+00],
         [ 1.4026e-01],
         [ 1.3590e+00],
         [-2.7623e-01],
         [ 6.8790e-01],
         [ 4.7019e-01],
         [ 1.1830e+00],
         [ 2.5140e-01],
         [ 1.9567e-01],
         [ 1.4597e+00],
         [ 1.3383e+00],
         [-5.3759e-01],
         [ 5.4988e-01],
         [ 7.0733e-01],
         [-1.1598e-01],
         [ 7.1554e-01],
         [ 1.3418e+00],
         [ 1.6129e+00],
         [ 8.1021e-01],
         [ 1.6692e+00],
         [ 4.7325e-01],
         [ 2.4416e-01],
         [ 8.3897e-01],
         [ 1.8066e+00],
         [ 1.0460e+00],
         [ 3.9240e-01],
         [-2.5766e-01],
         [ 5.2076e-01],
         [ 6.9135e-01],
         [ 8.308

In [172]:
output_squeeze = output_maxpool.squeeze(2)
print(output_squeeze.shape)
print("For a single value of the input series, we don't need one bracket for each learnt feature, so we squeeze out the bracket. Now the features of a single input value becomes one vector.")
output_squeeze

torch.Size([1, 320])
For a single value of the input series, we don't need one bracket for each learnt feature, so we squeeze out the bracket. Now the features of a single input value becomes one vector.


tensor([[ 7.1693e-01,  3.6379e-01,  6.9294e-01,  7.1109e-01,  4.5150e-01,
          5.8228e-01,  7.4167e-01, -1.8462e-02,  3.5803e-01,  1.2853e-02,
          8.3715e-01,  9.6151e-01,  1.3426e+00,  1.4026e-01,  1.3590e+00,
         -2.7623e-01,  6.8790e-01,  4.7019e-01,  1.1830e+00,  2.5140e-01,
          1.9567e-01,  1.4597e+00,  1.3383e+00, -5.3759e-01,  5.4988e-01,
          7.0733e-01, -1.1598e-01,  7.1554e-01,  1.3418e+00,  1.6129e+00,
          8.1021e-01,  1.6692e+00,  4.7325e-01,  2.4416e-01,  8.3897e-01,
          1.8066e+00,  1.0460e+00,  3.9240e-01, -2.5766e-01,  5.2076e-01,
          6.9135e-01,  8.3081e-01,  1.3088e+00,  1.8339e-01, -2.5171e-01,
          5.8868e-01,  1.4208e+00,  1.2572e+00,  3.3689e-01,  9.2980e-02,
          1.6370e-01,  8.2502e-01,  5.7390e-01,  1.6110e+00,  1.0459e+00,
          3.0195e-01,  1.0264e+00, -1.5252e-01,  1.2383e+00,  2.2057e-01,
          4.7698e-01,  6.1817e-01,  6.3510e-01,  6.7006e-01, -4.9933e-01,
         -4.6938e-02,  1.9735e+00,  1.

In [173]:
# user-defined parameter which is the length they want their latent representation to be
reduced_size = 160
out_channels=320
out_features = out_channels

# Fracheschi messed up the order of in_features and out_features. I switched to correct one here.
linear = torch.nn.Linear(in_features = out_channels, out_features = reduced_size)
output_linear = linear(output_squeeze)

print(output_linear.shape)
output_linear

torch.Size([1, 160])


tensor([[ 0.1652, -0.2739, -0.4565,  0.1729,  0.1193, -0.1135, -0.5626,  0.8112,
          0.2723,  0.3060,  0.0181, -0.0086, -0.7606,  0.3186,  0.0923, -0.6831,
         -0.2933,  0.8250, -0.2807,  0.7385,  0.4368, -0.3965, -0.1687, -0.0046,
         -0.1016,  0.0626,  0.0928, -0.1905, -0.2685, -0.5265,  0.1580, -0.2371,
         -0.4383, -0.0768,  0.4006,  0.3921,  0.0901,  0.0643, -0.6124,  0.3119,
         -0.0433, -0.3946,  0.7506, -0.2220, -0.0543, -0.1472, -0.7048,  0.0361,
         -0.4056,  0.2044, -0.9856,  0.8480,  0.4775, -0.3042,  0.3070, -0.2199,
         -0.2530,  0.3329, -0.1046, -0.0391, -0.1909,  0.4528,  0.1514,  0.8546,
          0.1549,  0.1124, -0.1357, -0.4681,  0.5549, -0.6133,  0.6729,  0.2290,
         -0.2853, -0.5276, -0.6911, -0.1877,  0.1283, -1.1351,  0.0426, -0.1646,
          0.0411, -0.5501, -0.2193,  1.1155, -0.0598, -0.2031,  0.4697, -0.0531,
          0.3671, -0.3642, -0.7328, -0.5127,  0.1166,  0.6081, -1.3057, -0.1706,
         -0.4864, -0.3666, -

# When there is NaN value in the input series
AdaptiveMaxPool1d()will just return NaN for all inputs. <br>
This behavior is discussed and coded via:https://github.com/pytorch/pytorch/issues/7645

In [57]:
print("There are several Conv1 layers in the whole network. All of the conv1's out_channels=40 except the last conv1's out_channel=320.")
conv1 = torch.nn.Conv1d(in_channels=1, out_channels=320, kernel_size=3, padding=2, dilation=1)
conv1

There are several Conv1 layers in the whole network. All of the conv1's out_channels=40 except the last conv1's out_channel=320.


Conv1d(1, 320, kernel_size=(3,), stride=(1,), padding=(2,))

In [41]:
def chomp1D(x, chomp_size):
    return x[:, :, :-chomp_size]

In [86]:
x_withNaN = torch.randn(1, 1, 10)
print("Assume we input a time series, [batchSize= 1, column number = 1, length of the series = 10]")
x_withNaN

Assume we input a time series, [batchSize= 1, column number = 1, length of the series = 10]


tensor([[[-0.5653, -0.7242, -1.2605,  2.4317, -0.9420, -0.6881,  0.2517,
           0.3291, -0.3225, -0.8670]]])

In [89]:
# set the 2nd value to be NaN
x_withNaN[0,0,1] = torch.log(torch.tensor([-1.]))
x_withNaN

tensor([[[-0.5653,     nan, -1.2605,  2.4317, -0.9420, -0.6881,  0.2517,
           0.3291, -0.3225, -0.8670]]])

In [90]:
output_conv1 = conv1(x_withNaN)
print(output_conv1.shape)
output_conv1

torch.Size([1, 320, 12])


tensor([[[ 0.5470,     nan,     nan,  ...,  0.7267,  0.4146,  0.2507],
         [ 0.0532,     nan,     nan,  ...,  0.4395, -0.0953, -0.5949],
         [-0.1594,     nan,     nan,  ..., -0.2250, -0.0742,  0.0822],
         ...,
         [ 0.3751,     nan,     nan,  ...,  0.4848,  0.5070,  0.2973],
         [ 0.1263,     nan,     nan,  ...,  0.2080,  0.0792, -0.1374],
         [ 0.0449,     nan,     nan,  ...,  0.3547,  0.3228, -0.2789]]],
       grad_fn=<SqueezeBackward1>)

In [91]:
output_chomp1D = chomp1D(output_conv1, 2)
print(output_chomp1D.shape)
output_chomp1D

torch.Size([1, 320, 10])


tensor([[[ 0.5470,     nan,     nan,  ...,  0.1292,  0.4548,  0.7267],
         [ 0.0532,     nan,     nan,  ..., -0.7530, -0.0552,  0.4395],
         [-0.1594,     nan,     nan,  ..., -0.0361, -0.2966, -0.2250],
         ...,
         [ 0.3751,     nan,     nan,  ...,  0.2157,  0.2863,  0.4848],
         [ 0.1263,     nan,     nan,  ..., -0.0043,  0.2757,  0.2080],
         [ 0.0449,     nan,     nan,  ..., -0.4247, -0.1080,  0.3547]]],
       grad_fn=<SliceBackward>)

In [92]:
reduce_size = torch.nn.AdaptiveMaxPool1d(1)

output_maxpool = reduce_size(output_chomp1D)

print(output_maxpool.shape)
print("As we can see, each input single value turn into 320 values, and these are the important features of this single value. The length of the input series is disregarded, ie 10 becomes 1. So now the input series is represented by 1 number that has 320 features.")
output_maxpool

torch.Size([1, 320, 1])
As we can see, each input single value turn into 320 values, and these are the important features of this single value. The length of the input series is disregarded, ie 10 becomes 1. So now the input series is represented by 1 number that has 320 features.


tensor([[[nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
        

In [93]:
output_squeeze = output_maxpool.squeeze(2)
print(output_squeeze.shape)
print("For a single value of the input series, we don't need one bracket for each learnt feature, so we squeeze out the bracket. Now the features of a single input value becomes one vector.")
output_squeeze

torch.Size([1, 320])
For a single value of the input series, we don't need one bracket for each learnt feature, so we squeeze out the bracket. Now the features of a single input value becomes one vector.


tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan

In [94]:
# user-defined parameter which is the length they want their latent representation to be
reduced_size = 160
out_channels=320
out_features = out_channels

# Fracheschi messed up the order of in_features and out_features. I switched to correct one here.
linear = torch.nn.Linear(in_features = out_channels, out_features = reduced_size)
output_linear = linear(output_squeeze)

print(output_linear.shape)
output_linear

torch.Size([1, 160])


tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
       grad_fn=<AddmmBackward>)

# Unequal lengths vs Same length training set

## Same length example

In [161]:
x = torch.randn(3, 2, 4)
x[:, 0, 2:]=numpy.nan
print("Assume we input a time series, [batchSize= 3, column number = 2, length of the series = 4]")
print("As we can see, for each training set, the corresponding series all have same lengths. ie. all col1 has length 2, all col2 has length 4")
x

Assume we input a time series, [batchSize= 3, column number = 2, length of the series = 4]
As we can see, for each training set, the corresponding series all have same lengths. ie. all col1 has length 2, all col2 has length 4


tensor([[[-0.8544, -1.0665,     nan,     nan],
         [ 0.0670, -0.0107,  0.7018,  0.3420]],

        [[ 0.6328, -0.2316,     nan,     nan],
         [ 0.6583,  0.9593,  0.7728, -1.2354]],

        [[-0.7437,  0.7021,     nan,     nan],
         [-1.9788, -0.1099,  1.3456,  0.8521]]])

## Unequal lengths example:

In [163]:
x = torch.randn(3, 2, 4)
x[0, 0, 3]=numpy.nan
x[1, 1, 2:]=numpy.nan
print("Assume we input a time series, [batchSize= 3, column number = 2, length of the series = 4]")
print("The corresponding columns of training sets do not have the same length")
x

Assume we input a time series, [batchSize= 3, column number = 2, length of the series = 4]
The corresponding columns of training sets do not have the same length


tensor([[[ 0.7168,  0.8649,  0.6129,     nan],
         [ 1.4183,  1.7877, -1.1541,  0.0053]],

        [[ 0.3740, -0.1801, -0.3869, -0.6879],
         [ 1.0990,  0.5861,     nan,     nan]],

        [[ 0.1607, -1.2338, -1.3170,  0.3218],
         [ 1.6872, -0.7801,  1.5478, -2.2525]]])

In [164]:
# scikit_wrappers.fit_encoder(X) will detect we have unequal lengths time series
varying = bool(numpy.isnan(numpy.sum(x.numpy())))
varying

True

Then `scikit_wrappers.fit_encoder(X`) will set:<br>
`loss = loss_varying( batch, self.encoder, train, save_memory=save_memory )`<br>
where `loss_varying = triplet_loss.TripletLossVaryingLength(compared_length, nb_random_samples, negative_penalty)`<br>
The need to distinguish between varying length from same length training sets is because we need to choose samples that are within series length (NaN not included), and when lengths are varying, we need to choose different lengths accordingly.

# Challenges 
1. Preserves only temporal order of values but not exact time of a value.
2. Cannot handle missing value in between the time series e.g. [2, 4, 5, NaN, 8, 0]. Only accepts time series where missing values are at begining or end.
3. Due to (1) and (2) above, it assumes equidistant time between values.

## Solution 1 - keep embedding method, change anomaly detection method
Use this embedding method and disregard exact time information, and a different anomaly detection method is needed.
#### Modify current anomaly detection algorithm:
Instead of using time interval to decide whether a value belongs or not, we can use a window of neighbor count.
#### Other anomaly algorithms:
- One-Class Support Vector Machine (Trained on normal data)
- Outlier detection with Local Outlier Factor (Computes local density of data)
- Kernel Density Estimation

## Solution 2 - change embedding method
1. Use call gaps to engineer a dataset only contains 'normal' sensor data.
2. Detect anomalies using method in paper "Learning Neural Representations for Network Anomaly Detection"

In [1]:
import os
import sys
sys.path.append(os.getcwd())

In [3]:
sys.path

['C:\\Users\\Paws\\Desktop\\MasterProject\\Franceschi',
 'C:\\Users\\Paws\\Anaconda3\\envs\\python3_8\\python38.zip',
 'C:\\Users\\Paws\\Anaconda3\\envs\\python3_8\\DLLs',
 'C:\\Users\\Paws\\Anaconda3\\envs\\python3_8\\lib',
 'C:\\Users\\Paws\\Anaconda3\\envs\\python3_8',
 '',
 'C:\\Users\\Paws\\Anaconda3\\envs\\python3_8\\lib\\site-packages',
 'C:\\Users\\Paws\\Anaconda3\\envs\\python3_8\\lib\\site-packages\\win32',
 'C:\\Users\\Paws\\Anaconda3\\envs\\python3_8\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Paws\\Anaconda3\\envs\\python3_8\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\Paws\\Anaconda3\\envs\\python3_8\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Paws\\.ipython',
 'C:\\Users\\Paws\\Desktop\\MasterProject\\Franceschi']

In [4]:
os.getcwd()

'C:\\Users\\Paws\\Desktop\\MasterProject\\Franceschi'